In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

ModuleNotFoundError: No module named 'keras'

In [4]:
classes = 2
img_rows, img_cols = 100, 100
batch_size = 16

In [5]:
train_dir = "/home/rupesh/AI/OpenCV/chest-xray-pneumonia/chest_xray/train"
valid_dir = "/home/rupesh/AI/OpenCV/chest-xray-pneumonia/chest_xray/val"

In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True)

validation_generator = validation_datagen.flow_from_directory(
        valid_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [7]:
# Classification Model Designing
model = Sequential()

#First set of conv-relu layer
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape= (img_rows, img_cols, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

#Second set of conv-relu layer
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(classes))
model.add(Activation('softmax'))


print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 49, 49, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 64)      

In [8]:
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("/home/rupesh/AI/OpenCV/Monkey Dataset-20200212T043806Z-001/Monkey Dataset/10-monkey-species/monkey_labels.txt",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0.001, 
                          patience = 3, 
                          verbose = 1,
                          restore_best_weights = True) 

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.001,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.0001)

callbacks = [earlystop, checkpoint, reduce_lr]

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 41322
nb_validation_samples = 13877
epochs = 3

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)


Epoch 1/3
2582/2582 [==============================] - 4409s 2s/step - loss: 1.1050 - accuracy: 0.6091 - val_loss: 0.6045 - val_accuracy: 0.6728

Epoch 00001: val_loss improved from inf to 0.60452, saving model to /home/rupesh/AI/OpenCV/Monkey Dataset-20200212T043806Z-001/Monkey Dataset/10-monkey-species/monkey_labels.txt
Epoch 2/3
2582/2582 [==============================] - 3172s 1s/step - loss: 0.5085 - accuracy: 0.8261 - val_loss: 0.2264 - val_accuracy: 0.7941

Epoch 00002: val_loss improved from 0.60452 to 0.22645, saving model to /home/rupesh/AI/OpenCV/Monkey Dataset-20200212T043806Z-001/Monkey Dataset/10-monkey-species/monkey_labels.txt
Epoch 3/3
2582/2582 [==============================] - 3466s 1s/step - loss: 0.3480 - accuracy: 0.8837 - val_loss: 0.2597 - val_accuracy: 0.7757

Epoch 00003: val_loss did not improve from 0.22645


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True)

validation_generator = validation_datagen.flow_from_directory(
        valid_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

In [1]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np


# image folder
folder_path = 'fruits-360/fruits-360/test/'
# path to model
model_path = 'fruits-360/fruit_V1.h5'
# dimensions of images
img_width, img_height = 100, 100
img = ""
# load the trained model
model = load_model(model_path)
# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# load all images into a list
# images = []
# for img in os.listdir(folder_path):
#     img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    

# # stack up images list to pass for prediction

classes = model.predict_classes(img, batch_size=16)
print(classes)
# x =classes[0]
# a = {0: 'Apple Braeburn', 1: 'Apple Golden 1', 2: 'Apple Golden 2', 3: 'Apple Golden 3', 4: 'Apple Granny Smith', 5: 'Apple Red 1', 6: 'Apple Red 2', 7: 'Apple Red 3', 8: 'Apple Red Delicious', 9: 'Apple Red Yellow', 10: 'Apricot', 11: 'Avocado', 12: 'Avocado ripe', 13: 'Banana', 14: 'Banana Red', 15: 'Cactus fruit', 16: 'Cantaloupe 1', 17: 'Cantaloupe 2', 18: 'Carambula', 19: 'Cherry 1', 20: 'Cherry 2', 21: 'Cherry Rainier', 22: 'Cherry Wax Black', 23: 'Cherry Wax Red', 24: 'Cherry Wax Yellow', 25: 'Clementine', 26: 'Cocos', 27: 'Dates', 28: 'Granadilla', 29: 'Grape Pink', 30: 'Grape White', 31: 'Grape White 2', 32: 'Grapefruit Pink', 33: 'Grapefruit White', 34: 'Guava', 35: 'Huckleberry', 36: 'Kaki', 37: 'Kiwi', 38: 'Kumquats', 39: 'Lemon', 40: 'Lemon Meyer', 41: 'Limes', 42: 'Lychee', 43: 'Mandarine', 44: 'Mango', 45: 'Maracuja', 46: 'Melon Piel de Sapo', 47: 'Mulberry', 48: 'Nectarine', 49: 'Orange', 50: 'Papaya', 51: 'Passion Fruit', 52: 'Peach', 53: 'Peach Flat', 54: 'Pear', 55: 'Pear Abate', 56: 'Pear Monster', 57: 'Pear Williams', 58: 'Pepino', 59: 'Physalis', 60: 'Physalis with Husk', 61: 'Pineapple', 62: 'Pineapple Mini', 63: 'Pitahaya Red', 64: 'Plum', 65: 'Pomegranate', 66: 'Quince', 67: 'Rambutan', 68: 'Raspberry', 69: 'Salak', 70: 'Strawberry', 71: 'Strawberry Wedge', 72: 'Tamarillo', 73: 'Tangelo', 74: 'Tomato 1', 75: 'Tomato 2', 76: 'Tomato 3', 77: 'Tomato 4', 78: 'Tomato Cherry Red', 79: 'Tomato Maroon', 80: 'Walnut'}
# print(a[x])

AttributeError: module 'numpy' has no attribute 'dtype'

In [9]:
import numpy as np
import keras
from keras.preprocessing import image
from keras.models import load_model

classifier = load_model('/home/rupesh/AI/OpenCV/Monkey Dataset-20200212T043806Z-001/Monkey Dataset/10-monkey-species/monkey_labels.h5')
test_image = image.load_img('/home/rupesh/AI/OpenCV/Monkey Dataset-20200212T043806Z-001/Monkey Dataset/monkey.jpeg',
target_size = (100,100,3))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis = 0)
result = classifier.predict(test_image)
print(result) 

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
